# Series 3, Online Convex Programming

In [14]:
import math

import numpy as np
import pandas as pd

from scipy.linalg import norm
from sklearn.base import BaseEstimator, ClassifierMixin

%matplotlib inline

import matplotlib.pyplot as plt

In [2]:
# Ensure consistency across runs.
np.random.seed(1337)

In [3]:
Xtrain = np.genfromtxt('data/Xtrain.csv', delimiter=',')
Ytrain = np.genfromtxt('data/Ytrain.csv', delimiter=',', dtype='int8')
Xtest = np.genfromtxt('data/Xtest.csv', delimiter=',')
Ytest = np.genfromtxt('data/Ytest.csv', delimiter=',', dtype='int8')

In [4]:
def permute_data(x, y):
    """Shuffles both numpy arrays in unison."""
    perm = np.random.permutation(x.shape[0])
    return x[perm, :], y[perm]

Xtrain, Ytrain = permute_data(Xtrain, Ytrain)
Xtest, Ytest = permute_data(Xtest, Ytest)

In [5]:
from sklearn.utils.estimator_checks import check_estimator

class OnlineSVMClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, **params):
        self.w = None
        self.lbd = 1.0
        self.set_params(**params)
    
    def fit(self, X, y):
        self.w = np.zeros(X.shape[1], dtype='float64')
        
        for t, (x, label) in enumerate(zip(X, y)):
            eta = 1.0 / np.sqrt(t + 1)
            hinge = label * np.inner(self.w, x)
            if hinge < 1:
                self.w = self.w + eta * label * x
                self.project()


        return self
    
    def project(self):
        sqrt_lambda = np.sqrt(self.lbd)
        w_norm = norm(self.w)
        regularizer = 1.0 / (sqrt_lambda * w_norm)
        self.w *= min(1.0, regularizer)
    
    def predict(self, X):
        signs = np.sign(np.inner(self.w, X))
        signs[signs == 0] = -1
        return signs.astype('int8')
    
    def get_params(self, deep=True):
        return {"lbd": self.lbd}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
# check_estimator(OnlineSVMClassifier)

In [6]:
cls = OnlineSVMClassifier()

In [9]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

parameters = {
    'lbd': [0.001, 0.005, 0.0075, 0.01, 0.0125, 0.05, 0.1]
}
gs = GridSearchCV(cls, parameters)
gs_result = gs.fit(Xtrain, Ytrain)

print("Best score: %f" % gs_result.best_score_)
print("Best score params: %s" % gs_result.best_params_)

Best score: 0.936115
Best score params: {'lbd': 0.01}


In [10]:
import scipy.stats as stats

rs_params = {
    "lbd": stats.uniform(loc=0.001, scale=0.099)
}
rs_n_iter = 100
rs = RandomizedSearchCV(cls, rs_params, rs_n_iter, n_jobs=1)
rs_result = rs.fit(Xtrain, Ytrain)

print("Best score: %f" % rs_result.best_score_)
print("Best score params: %s" % rs_result.best_params_)

Best score: 0.937240
Best score params: {'lbd': 0.011353224786015938}


In [12]:
rs_best = rs_result.best_estimator_

test_score = rs_best.score(Xtest, Ytest)
print("Best test score: %f" % test_score)

Best test score: 0.955836


In [36]:
test_count = Xtrain.shape[0]

cls = OnlineSVMClassifier(lbd=0.11)

# TODO(andrei) Plot this shit.
# TODO(andrei) Logistic regression with tonsa comments.
# TODO(andrei) Try to get a general idea of how they implemented the projection to the L1-ball (i.e. LASSO-like).
for amount in list(np.round((np.logspace(0, np.log10(test_count), 30)))):
    Xsubsample = Xtrain[:int(amount),:]
    Ysubsample = Ytrain[:int(amount)]
    cls.fit(Xsubsample, Ysubsample)
    print("%d training samples: %f" % (amount, cls.score(Xtest, Ytest)))

1 training samples: 0.468980
1 training samples: 0.468980
2 training samples: 0.468980
3 training samples: 0.531020
4 training samples: 0.531020
5 training samples: 0.472135
7 training samples: 0.555205
10 training samples: 0.555205
13 training samples: 0.740799
18 training samples: 0.669295
25 training samples: 0.741851
35 training samples: 0.584122
48 training samples: 0.669821
66 training samples: 0.701367
91 training samples: 0.855941
126 training samples: 0.904837
174 training samples: 0.772345
241 training samples: 0.906414
332 training samples: 0.901157
459 training samples: 0.886961
634 training samples: 0.917455
875 training samples: 0.943218
1208 training samples: 0.937960
1668 training samples: 0.945321
2302 training samples: 0.954784
3179 training samples: 0.948475
4389 training samples: 0.949001
6060 training samples: 0.944269
8367 training samples: 0.942166
11552 training samples: 0.954784
